In [3]:
import os
os.chdir(os.getcwd() + '\\code\\source_dir_v2')
print(os.getcwd())
from copy import deepcopy
from pathlib import Path
import numpy as np
import random
import pandas as pd
import argparse
import torch
# from callbacks import CSVParamLogger
from scipy.stats import spearmanr, rankdata
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from tqdm import tqdm, trange
import time
from transformers import BertConfig, BertForPreTraining
from transformers import AdamW, BertTokenizer, get_linear_schedule_with_warmup

# from apex import amp
# from apex.parallel import DistributedDataParallel as DDP
# from apex.optimizers import FusedAdam
# from apex.normalization.fused_layer_norm import FusedLayerNorm
import logging
import warnings
#import horovod.torch as hvd

warnings.filterwarnings("ignore")
from model_dataset import QuestDataset
from utils.loss_function import *
from utils.metric import *
from utils.lrs_scheduler import *
from utils.file import *

C:\Users\admin\Documents\Nitin\mycodes\kaggle_google_quest_qna\lmtraining\distributed_v3\code\source_dir_v2
matplotlib.get_backend :  module://ipykernel.pylab.backend_inline


In [7]:
stackx_data.shape

(600000, 15)

In [7]:
os.getcwd()

'C:\\Users\\admin\\Documents\\Nitin\\mycodes\\kaggle_google_quest_qna\\lmtraining\\distributed_v3\\code\\source_dir_v2'

In [8]:
stackx_data = pd.read_csv( "C:/Users/admin/Documents/Nitin/mycodes/kaggle_google_quest_qna/lmtraining/distributed_v3/data/input/qa_stackexchange_cleaned.csv", nrows=20000)
TARGETS = ['question_score','question_views','question_favs','answer_score','is_answer_accepted']
stackx_data["question_title"] = stackx_data["question_title"].astype(str)
stackx_data["question_body"] = stackx_data["question_body"].astype(str)
stackx_data["answer"] = stackx_data["answer"].astype(str)
# print(stackx_data.shape)


# Normalize aux targets
encoded = []
trange = tqdm(stackx_data["host"].unique())
for host in trange:
    host_mask = stackx_data["host"] == host
    trange.set_description(str(host))
    host_labels = deepcopy(stackx_data[host_mask][TARGETS])
    for col in ["question_score", "question_views", "question_favs", "answer_score"]:
        host_labels[col] = rankdata(stackx_data[host_mask][col]) / host_mask.sum()
    encoded.append(host_labels)

encoded = pd.concat(encoded, sort=False).reindex(stackx_data.index)
stackx_data[encoded.columns] = encoded


# Train-Val Split
train_df, test_df = train_test_split(stackx_data, test_size=0.1, random_state=44)

arduino.stackexchange.com: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.32it/s]


In [9]:
train_df.columns

Index(['id', 'host', 'question_username', 'question_score', 'question_views',
       'question_favs', 'answers_count', 'answers_max_score',
       'answers_mean_score', 'question_title', 'question_body',
       'answer_username', 'answer', 'answer_score', 'is_answer_accepted'],
      dtype='object')

In [10]:
class QuestMLMDataset(QuestDataset):
    def __init__(self, df, tokenizer, max_len=128, content="Question_Answer", mlm_prob=0.15,
                 non_masked_idx=-1, sop_prob=0.5, target_cols=TARGETS):
        super(QuestMLMDataset, self).__init__(df=df, target_cols=target_cols, max_len=max_len, content=content,
                                              tokenizer=tokenizer)
        self.mlm_probability = mlm_prob
        self.sop_prob = sop_prob
        self.non_masked_idx = non_masked_idx
        self.tokenizer = tokenizer
        self.cls_token_idx = tokenizer.convert_tokens_to_ids(tokenizer.cls_token)
        self.sep_token_idx = tokenizer.convert_tokens_to_ids(tokenizer.sep_token)
        self.mask_token_idx = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    def _mask_tokens(self, inputs, masked_random_replace_prob=0.2):
        labels = inputs.clone()
        masked_indices = torch.bernoulli(torch.full(labels.shape, self.mlm_probability)).bool()
        for special_tokens in [self.cls_token_idx, self.sep_token_idx, self.mask_token_idx]:
            masked_indices &= inputs != special_tokens
        labels[~masked_indices] = self.non_masked_idx
        indices_replaced = (
                    torch.bernoulli(torch.full(labels.shape, 1 - masked_random_replace_prob)).bool() & masked_indices)
        inputs[indices_replaced] = self.mask_token_idx
        indices_random = (torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced)
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]
        return inputs, labels

    def __getitem__(self, index):
        title, body, answer = self._get_text(index)
        numeric_targets = self.get_label(index)
        sop_label = 0
        if np.random.uniform(0, 1) < self.sop_prob:
            sop_label = 1
            perm = list(np.random.permutation(range(3)))
            if perm == [0, 1, 2]:
                perm = [0, 2, 1]
            title, body, answer = [[title, body, answer][i] for i in perm]
        input_ids, token_type_ids, attention_mask = self.get_token_ids(title, body, answer)
        input_ids, token_type_ids, attention_mask = map(torch.LongTensor, [input_ids, token_type_ids, attention_mask])
        input_ids, labels = self._mask_tokens(torch.LongTensor(input_ids))
        return ((input_ids, token_type_ids, attention_mask), (labels, sop_label, numeric_targets))


class BertPretrain(BertForPreTraining):
    def __init__(self, config, num_labels):
        super(BertPretrain, self).__init__(config, )
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)

    def forward(self, input_ids=None, token_type_ids=None, attention_mask=None):
        outputs = self.bert(input_ids, token_type_ids, attention_mask)
        sequence_output, pooled_output = outputs[:2]
        logits = self.classifier(self.dropout(torch.mean(sequence_output, dim=1)))
        pred_scores, seq_rel_score = self.cls(sequence_output, pooled_output)
        outputs = (pred_scores, seq_rel_score, logits,pooled_output)
        return outputs


def spearmanr(y_true, y_pred):
    corr = [spearmanr(pred_col, act_col).correlation for pred_col, act_col in zip(y_pred.T, y_true.T)]
    return np.nanmean(corr)


# Loss
class MLMloss(nn.CrossEntropyLoss):
    def forward(self, logits, targets):
        n_samples = np.prod(targets.shape)
        loss = super(MLMloss, self).forward(logits.view(n_samples, -1), targets.view(n_samples))
        return loss


class SOPloss(nn.CrossEntropyLoss):
    def forward(self, logits, targets):
        n_samples = np.prod(targets.shape)
        loss = super(SOPloss, self).forward(logits.view(-1, 2), targets.view(-1))
        return loss


class PretrainingLoss(torch.nn.Module):
    def __init__(self, targets_alpha=1.0):
        super(PretrainingLoss, self).__init__()
        self.mlm_loss = MLMloss(ignore_index=-1)
        self.sop_loss = SOPloss()
        self.bce = torch.nn.BCEWithLogitsLoss()
        self.targets_alpha = targets_alpha

    def forward(self, logits, targets):
        return (
                self.mlm_loss(logits[0], targets[0])
                + self.sop_loss(logits[1], targets[1])
                + self.targets_alpha * self.bce(logits[2], targets[2])
        )


class MLMPerplexity(MLMloss):
    __name__ = "mlm_perplexity"

    def forward(self, logits, targets):
        logits, targets = logits[0], targets[0]
        loss = super(MLMPerplexity, self).forward(logits, targets)
        perplexity = 2 ** loss
        return float(perplexity)


def sop_accuracy(logits, targets):
    logits, targets = logits[1], targets[1]
    pred = torch.argmax(logits.view(-1, 2), dim=-1)
    targets = targets.view(-1)
    return float(torch.mean((pred == targets).float()))

In [11]:
tokenizer = BertTokenizer(str("C:/Users/admin/Documents/Nitin/mycodes/kaggle_google_quest_qna/lmtraining/distributed_v3/data/input/data/vocab.txt"), do_basic_tokenize=True, do_lower_case=False)
train_dataset = QuestMLMDataset(train_df, tokenizer, target_cols=TARGETS)
val_dataset = QuestMLMDataset(test_df, tokenizer, target_cols=TARGETS)
config = BertConfig.from_json_file(str( "C:/Users/admin/Documents/Nitin/mycodes/kaggle_google_quest_qna/lmtraining/distributed_v3/data/input/data/config.json"))
model = BertPretrain(config, len(TARGETS))

In [12]:
orig_bert = BertForPreTraining.from_pretrained("bert-base-cased")
orig_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
state_dict = orig_bert.state_dict()
del state_dict["cls.predictions.decoder.weight"], state_dict["cls.predictions.bias"], state_dict[
    "cls.predictions.decoder.bias"]
orig_embedding = state_dict["bert.embeddings.word_embeddings.weight"]
extra_tokens = list(tokenizer.vocab.keys())[len(orig_tokenizer.vocab):]
new_tokens_as_orig_indices = [[i] for i in range(len(orig_tokenizer.vocab))] + [
    orig_tokenizer.encode(t, add_special_tokens=False) for t in extra_tokens]
new_embedding = torch.zeros(len(new_tokens_as_orig_indices), orig_embedding.shape[-1])
new_embedding.normal_(mean=0.0, std=0.02)
for row, indices in enumerate(new_tokens_as_orig_indices):
    if len(indices) > 0:
        new_embedding[row] = orig_embedding[indices].mean(0)
state_dict["bert.embeddings.word_embeddings.weight"] = new_embedding
model.load_state_dict(state_dict, strict=False)
model.tie_weights()



train_loader = DataLoader(train_dataset, batch_size=8)
#Keeping validation as non-distributed
val_loader = DataLoader(val_dataset, batch_size=8)

I0514 23:46:02.075158  2844 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at C:\Users\admin\.cache\torch\transformers\b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
I0514 23:46:02.080227  2844 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "mod

In [14]:
%%time
for tr_batch_i, (inputs, targets) in enumerate(train_loader):
    if tr_batch_i==0:
        print('tr_batch_i')
        input_ids, token_type_ids, attention_mask = inputs
        targets_mlm, targets_sop, targets_extra = targets
        logits = model(input_ids, token_type_ids, attention_mask)
        logits_mlm, logits_sop, logits_extra, pooled = logits
    else:
        continue
    

tr_batch_i


In [15]:
pooled.shape

torch.Size([8, 768])

In [16]:
input_ids

tensor([[  101, 38071, 62599,  ...,  1115,   119,   102],
        [  101, 59921,   103,  ...,  1208,   119,   102],
        [  101, 14709,   170,  ...,   114,   119,   102],
        ...,
        [  101, 11193,   103,  ...,   108, 38262,   102],
        [  101,   146,  1821,  ..., 51318,   136,   102],
        [  101, 32492,   103,  ...,  1141,   136,   102]])

In [17]:
mlm_loss = MLMloss(ignore_index=-1)(logits_mlm, targets_mlm).item()
sop_loss = SOPloss()(logits_sop, targets_sop).item()
mlm_perplexity = MLMPerplexity(ignore_index=-1)((logits_mlm, logits_sop, logits_extra),(targets_mlm, targets_sop, targets_extra))
sop_acc = sop_accuracy((logits_mlm, logits_sop, logits_extra),(targets_mlm, targets_sop,targets_extra))

In [29]:
print(targets_mlm.shape)
print(targets_sop.shape)
print(targets_extra.shape)
print(input_ids.shape)
print(token_type_ids.shape)
print(attention_mask.shape)
print(mlm_loss)
print(sop_loss)
print(mlm_perplexity)
print(sop_acc)
print(logits_mlm.shape)
print(logits_sop.shape)
print(logits_extra.shape)

torch.Size([8, 128])
torch.Size([8])
torch.Size([8, 5])
torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 128])
9.34911823272705
1.567033052444458
652.1763305664062
0.625
torch.Size([8, 128, 110000])
torch.Size([8, 2])
torch.Size([8, 5])


In [19]:
targets_sop

tensor([0, 0, 0, 1, 0, 0, 1, 1])

In [24]:
logits_sop

tensor([[ 2.9986, -2.2750],
        [ 2.6503, -1.8422],
        [ 2.0961, -1.0245],
        [ 2.1254, -1.1128],
        [ 1.6194, -0.4749],
        [ 3.0544, -2.1734],
        [ 2.1251, -1.1417],
        [ 3.2816, -2.4898]], grad_fn=<AddmmBackward>)

In [31]:
w = model.cls.seq_relationship.state_dict()['weight']
b = model.cls.seq_relationship.state_dict()['bias']
print(w.shape)
print(b.shape)

torch.Size([2, 768])
torch.Size([2])


In [32]:
x=torch.rand((1,768))
output = torch.mm(pooled,torch.transpose(w,0,1)) + b
output


tensor([[ 2.9986, -2.2750],
        [ 2.6503, -1.8422],
        [ 2.0961, -1.0245],
        [ 2.1254, -1.1128],
        [ 1.6194, -0.4749],
        [ 3.0544, -2.1734],
        [ 2.1251, -1.1417],
        [ 3.2816, -2.4898]], grad_fn=<AddBackward0>)

In [27]:
for i in range(pooled.shape[0]):
    print(pooled[i,:])

tensor([-0.6431,  0.5832,  0.9989, -0.8948,  0.7898, -0.3820,  0.8346, -0.8237,
        -0.6892, -0.4968,  0.8630,  0.9191, -0.8367, -0.9983, -0.7681, -0.7107,
         0.8587, -0.5896, -0.9994,  0.4606,  0.0474, -0.9979,  0.5147,  0.2998,
         0.6597, -0.1198,  0.8687,  0.9991,  0.5288,  0.2529,  0.1140, -0.8886,
        -0.2462, -0.9789,  0.2963, -0.2580, -0.1181, -0.2651, -0.3137, -0.0101,
        -0.1880,  0.5700, -0.0327, -0.4400, -0.1142, -0.1373,  0.2286, -0.1764,
        -0.2754,  0.9999, -0.8045,  0.9998, -0.8261,  0.9822,  0.8720,  0.1916,
         0.8933,  0.1491, -0.8682,  0.2915,  0.6768,  0.0044,  0.6759, -0.3878,
        -0.5226,  0.6660, -0.2957,  0.3238, -0.0159,  0.0717, -0.0425,  0.4768,
         0.8646, -0.7591,  0.0161, -0.7480,  0.7355, -0.9990,  0.8082,  0.9990,
         0.1002, -0.9904,  0.8975, -0.3334,  0.1221, -0.7445, -0.8191, -0.9893,
         0.1739, -0.2528, -0.1109, -0.8420, -0.1446,  0.4793,  0.9994, -0.1870,
        -0.3045,  0.2573,  0.0875,  0.18

In [25]:
torch.nn.CrossEntropyLoss(torch.tensor([[],[]]), torch.tensor())

0.625